# LSTM Models

Created: 2019.10.18
Notebook sequence: 8

For experimenting with keras lstm models

---

In [1]:
%load_ext autoreload
%autoreload 2

In [31]:
import util
import pandas as pd
import keras
import numpy as np
from keras.layers import LSTM, TimeDistributed, Dense
from sklearn.model_selection import train_test_split

In [8]:
data = util.load_dataset("os_reliable_w2v/sequence.pkl")
df = util.load_selection_dataset("os_reliability.csv")

In [23]:
y = list(df.reliable)
X = data
X_train, X_valid, Y_train, Y_valid = train_test_split(X,y, test_size=0.20, random_state=13)
X_train

In [34]:
model = keras.Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(None, 300)))
model.add(TimeDistributed(Dense(2, activation='sigmoid')))

In [37]:
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(X_train, Y_train)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 7481 arrays: [array([[-0.03759766,  0.02539062,  0.10253906, ...,  0.00124359,
         0.1328125 ,  0.30078125],
       [ 0.00704956, -0.07324219,  0.171875  , ...,  0.01123047,
         0.1640625 ,  0.10693359],...